# DataMining Project

## Description du projet
L'objectif de ce projet est de recommander des cartes Yu Gi Oh en fonction des préférences de l'utilisateur. 

Auteur : **Gorvien Mathis** _étudiant en 4ème année en CLBD_


Un grand merci à **Perrichet Theotime** _étudiant en 4ème année en Robotique_ pour son aide précieuse.

## Install libraries


In [ ]:
!pip install -r requirements.txt

## Import database

#### Authentification
_Pour utiliser l' API publique de Kaggle , vous devez d'abord vous authentifier à l'aide d'un jeton d'API. Depuis l'en-tête du site, cliquez sur votre photo de profil d'utilisateur, puis sur "Mon compte" dans le menu déroulant. Cela vous amènera aux paramètres de votre compte sur https://www.kaggle.com/account. Faites défiler jusqu'à la section de la page intitulée API :_

_Pour créer un nouveau jeton, cliquez sur le bouton "Créer un nouveau jeton API". Cela téléchargera un nouveau jeton d'authentification sur votre machine._

_Si vous utilisez l'outil Kaggle CLI, l'outil recherchera ce jeton dans __~/.kaggle/kaggle.json sous Linux__, OSX et d'autres systèmes d'exploitation basés sur UNIX, et dans __C:\Users<Windows-username>.kaggle\kaggle.json sous Windows__. Si le jeton n'est pas là, une erreur sera levée. Par conséquent, une fois que vous avez téléchargé le jeton, vous devez le déplacer de votre dossier Téléchargements vers ce dossier._

_Si vous utilisez directement l'API Kaggle, l'endroit où vous conservez le jeton n'a pas d'importance, tant que vous êtes en mesure de fournir vos informations d'identification au moment de l'exécution._

In [ ]:
import os
import kaggle
import shutil

In [ ]:

# Download the dataset
kaggle.api.authenticate()
kaggle.api.dataset_download_files('archanghosh/yugioh-database/Yugi_images', path='.', unzip=True, quiet=False, force=False)

In [ ]:
# Move the csv file to the data folder
shutil.move('./Yugi_db_cleaned.csv', './data_csv/')

# DataBase creation

## I. Create the Yu Gi Oh database

In [ ]:
from PIL import Image
import os
import json
from datetime import datetime
import numpy as np
import cv2
from tqdm import tqdm

In [ ]:
img_dir = r"./Yugi_images" # path to the folder that contains the images

# initialize the data dictionary
all_metadata = {}
color_data={}

In [ ]:
# Function allowing to crop the images in order to have more precise Kmeans results
def crop_and_resize_card(image_path, output_size=(224, 224)):
    # Load the image
    image = cv2.imread(image_path)
    
    # Calculate the central square coordinates
    height, width, _ = image.shape
    min_dim = min(height, width)
    x = (width - min_dim) // 2
    y = (height - min_dim) // 2
    
    # Crop the central square
    cropped_image = image[y+30:y+min_dim-70, x+50:x+min_dim-50]

    # Resize the image to the desired output size
    resized_image = cv2.resize(cropped_image, output_size)

    return resized_image

# Create the folder containing the cropped images
if not os.path.exists('./resized_yugioh'):
    os.makedirs('./resized_yugioh')

In [ ]:
# Boucle through all the image files in the directory
for img_filename in os.listdir(img_dir):
    if img_filename.endswith(".jpg") or img_filename.endswith(".png"):
        # build the full path to the image file
        img_path = os.path.join(img_dir, img_filename)

        # load the image file
        with Image.open(img_path) as img:

            # extract metadata from the image
            img_filename= img.filename
            img_format = img.format
            img_size = img.size
            img_orientation = "landscape" if img_size[0] > img_size[1] else "portrait"
            creation_date =  datetime.fromtimestamp(os.path.getctime(img_path)).strftime('%d/%m/%Y')

            # Split the filename on '/' and take the last part
            filename = img_filename.split('/')[-1]

            # Remove the file extension
            filename = filename.split('.')[0]

            # Crop the images
            resized_image = crop_and_resize_card(img_path)
            new_path = os.path.join('./resized_yugioh', filename + '.jpg')
            cv2.imwrite(new_path, resized_image)

            # Create a dictionary of metadata for this image
            metadata = {
                "id": filename,
                "format": img_format,
                "size": img_size,
                "orientation": img_orientation,
                "creation_date": creation_date,
                "tags": ""
            }

            # add the metadata of this image to the dictionary of all metadata
            all_metadata[filename] = metadata

In [ ]:
# Add the metadata to the JSON file
with open('database.json', "w") as f:
    json.dump(all_metadata, f, indent=4)

## II. Create the color database

In [ ]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from sklearn.cluster import MiniBatchKMeans
from webcolors import CSS3_NAMES_TO_HEX, hex_to_rgb, rgb_to_hex, hex_to_name
from tqdm import tqdm
import cv2

In [ ]:
def closest_color(requested_color):
    requested_color = hex_to_rgb(requested_color)
    min_colors = {}
    for key, value in CSS3_NAMES_TO_HEX.items():
        color = hex_to_rgb(value)
        red_difference = (color[0] - requested_color[0]) ** 2
        green_difference = (color[1] - requested_color[1]) ** 2
        blue_difference = (color[2] - requested_color[2]) ** 2
        difference = red_difference + green_difference + blue_difference
        min_colors[difference] = key
    return min_colors[min(min_colors.keys())]

In [ ]:
# Boucle through all the image files in the directory
color_data = {}
img_dir = r"./resized_yugioh" # On utilise les images recadrées
for img_filename in tqdm(os.listdir(img_dir)): 
    if img_filename.endswith(".jpg") or img_filename.endswith(".png"):
        # Build the full path to the image file
        img_path = os.path.join(img_dir, img_filename)
        # Open the image file
        with Image.open(img_path) as img:
            
            pixel_matrix = np.array(img) # Convert the image to a numpy array

            # Extract the R, G, B values, even if the image contains more than 3 channels (e.g. RGBA)
            if pixel_matrix.shape[2] == 4:
                pixel_matrix = pixel_matrix[:, :, :3]

            # Convert the pixel matrix to a pixel array
            pixel_data = pixel_matrix.reshape((-1, 3))

            # Use MiniBatchKMeans to find the largest cluster
            kmeans = MiniBatchKMeans(n_clusters=3, random_state=0, n_init=3).fit(pixel_data)
            dominant_color = kmeans.cluster_centers_[kmeans.predict([[0, 0, 0]])][0]

            # Convert the dominant color to hexadecimal
            hex_color = '#{:02x}{:02x}{:02x}'.format(int(dominant_color[0]), int(dominant_color[1]), int(dominant_color[2]))
            
            # Convert the hexadecimal value to a color name
            try:
                color_name = hex_to_name(hex_color)
            except ValueError:
                color_name = closest_color(hex_color)

            # Split the filename on '/' and take the last part
            filename = img_filename.split('/')[-1]

            # Remove the file extension
            filename = filename.split('.')[0]

            # create a dictionary of color for this image
            color = {
                "id": filename,
                "nom couleur": color_name
            }

            # add the color of this image to the dictionary of all colors
            color_data[filename] = color


In [ ]:
# Add the color to the JSON file
with open('color_data.json', 'w') as f:
    json.dump(color_data, f, indent=4)

## III. Add colors to Yu Gi Oh cards

In [ ]:
# Open the JSON files
with open('database.json', 'r') as f:
    yugioh_data = json.load(f)

with open('color_data.json', 'r') as f:
    color_data = json.load(f)

# If the id of the yugioh card is the same as the id of the color, add the color to the yugioh card
for yugioh in yugioh_data:
    for color in color_data:
        
        # Split the filename on '/' and take the last part
        filename = yugioh_data[yugioh]['id'].split('/')[-1]

        # Remove the file extension
        filename = filename.split('.')[0]

        if filename == color_data[color]['id']:
            yugioh_data[yugioh]['couleur dominante'] = color_data[color]['nom couleur']

with open('database.json', 'w') as f:
    json.dump(yugioh_data, f, indent=4)


## Add tags

In [ ]:
import pandas as pd
import json
import csv

In [ ]:
# Load the data from tags.json into data_d["tags"]
df = pd.read_csv(r"./data_csv/Yugi_db_cleaned.csv", sep=',', header=0)
df_selected = df.loc[:, ['Card_name','Card-set', 'Image_name', 'Rarity', 'Card type', 'Attribute','Types', 'Level']]

In [ ]:
# Rename the column Card_name to Card Name
df_selected.rename(columns={'Card_name': 'Card Name'}, inplace=True)

# Rename the column Card-set to Card Set
df_selected.rename(columns={'Card-set': 'Card Set'}, inplace=True)

# Rename the column Image_name to Card Number
df_selected.rename(columns={'Image_name': 'Card Number'}, inplace=True)

# Replace the .png in the Card Number column with an empty string
df_selected['Card Number'] = df_selected['Card Number'].str.replace('.png', '')

# Keep only the first value in the Types column
df_selected['Types'] = df_selected['Types'].str.split(' / ').str[0]

# Rename the column Types to Caracteristic
df_selected.rename(columns={'Types': 'Caracteristic'}, inplace=True)

with open('tags.json', 'w') as f:
    f.write(df_selected.to_json(orient='records')) 

with open('tags.json', 'r') as f:
    data_t = json.load(f)

with open('tags.json', 'w') as f:
    json.dump(data_t, f, indent=4)

with open('database.json', 'r') as f:
    data_d = json.load(f)

In [ ]:
# Load the data from tags.json into data_d["tags"]
for key in data_d:
    for i in range(len(data_t)):

        # Split the filename on '/' and take the last part
        filename = data_d[key]['id'].split('/')[-1]

        if filename == data_t[i]["Card Number"]:
            data_d[key]["tags"] = data_t[i]

# Delete the Card Number key from data_d["tags"]
for key in data_d:
    if data_d[key]["tags"] != '':
        data_d[key]["tags"].pop("Card Number")

# Save the data to database.json
with open('database.json', 'w') as f:
    json.dump(data_d, f, indent=4)

## Create user database

In [ ]:
import json
from random import randint, choice
import os

In [ ]:
# Function to filter images according to user preferences
def filter_images(images, CardType, Attribute, Rarity, Set):
    filtered_images = []
    for image in images.values():
        if len(image) == 7:
            if image["tags"] != '':
                color = image["couleur dominante"]
                type = image["tags"]["Card type"]
                attribute = image["tags"]["Attribute"]
                rarity = image["tags"]["Rarity"]
                set = image["tags"]["Card Set"]
                tags= image["tags"]

                # Verify that the image matches the user preferences
                if (set == Set) and (attribute == Attribute) and (type == CardType) and (rarity == Rarity):
                    tags["color"]=color
                    filtered_images.append(tags)
    return filtered_images

In [ ]:
# Get color name, card type, attribute, rarity, caracteristic and set from the user

data_t={}
with open("database.json", "r") as f:
    data=json.load(f)
    for i in data:
        data_t[data[i]["id"]]=data[i]

Type_t=[]
for i in data_t:
    if data_t[i]["tags"] != '':
        if data_t[i]["tags"]["Card type"] not in Type_t:
            Type_t.append(data_t[i]["tags"]["Card type"])

Attribute_t=[]
for i in data_t:
    if data_t[i]["tags"] != '':
        if data_t[i]["tags"]["Attribute"] not in Attribute_t:
            Attribute_t.append(data_t[i]["tags"]["Attribute"])

Rarity_t=[]
for i in data_t:
    if data_t[i]["tags"] != '':
        if data_t[i]["tags"]["Rarity"] not in Rarity_t:
            Rarity_t.append(data_t[i]["tags"]["Rarity"])

Set_t=[]
for i in data_t:
    if data_t[i]["tags"] != '':
        if data_t[i]["tags"]["Card Set"] not in Set_t:
            Set_t.append(data_t[i]["tags"]["Card Set"])


Caracteristic_t=[]
for i in data_t:
    if data_t[i]["tags"] != '':
        if data_t[i]["tags"]["Caracteristic"] not in Caracteristic_t:
            Caracteristic_t.append(data_t[i]["tags"]["Caracteristic"])

Level_t=[]
for i in data_t:
    if data_t[i]["tags"] != '':
        if data_t[i]["tags"]["Level"] not in Level_t:
            Level_t.append(data_t[i]["tags"]["Level"])

# Get the color name from database.json
color_t=[]
for i in data_t:
    if len(data_t[i]) == 7:
        if data_t[i]["couleur dominante"] not in color_t:
            color_t.append(data_t[i]["couleur dominante"])

legendary_t=[True, False]


In [ ]:
# Save the data to recommendation.json
with open("recommendation.json", "w") as f:
    json.dump({"Set":Set_t, "Attribute":Attribute_t, "Color":color_t, "Caracteristic":Caracteristic_t, "Rarity":Rarity_t}, f, indent=4)

In [ ]:
# Simulate the user's preferences

favorite_t=["NoFavorite", "Favorite"]
all_user={}
nb_user=100
i=0

while i<nb_user:
    result=[]
    Type=choice(Type_t) 
    Attribute=choice(Attribute_t) 
    Rarity=choice(Rarity_t) 
    Set=choice(Set_t) 
    Caracteristic=choice(Caracteristic_t) 
    Level=choice(Level_t) 
    data=filter_images(data_t, Type, Attribute, Rarity, Set)
    if len(data)>0:
        for k in range(len(data)): # choose favorite or not
            result.append(favorite_t[randint(0, len(favorite_t)-1)])
        all_user[i]={"data":data, "result":result}
        i+=1
    else:
        continue

    

In [ ]:
pok=[]
like=[]

# create a dictionary with the data and the results
for i in all_user:
    for k in range(len(all_user[i]["data"])):
        pok.append(all_user[i]["data"][k])
        like.append(all_user[i]["result"][k])

user={"data":pok, "result":like}

# count the number of favorite and not favorite
nb_favorite=0
nb_nofavorite=0
for i in user["result"]:
    if i=="Favorite":
        nb_favorite+=1
    else:
        nb_nofavorite+=1

print(len(user["result"]))
print("nb_favorite: ", nb_favorite)
print("nb_nofavorite: ", nb_nofavorite)

In [ ]:
# save the data to user.json
with open("user.json", "w") as f:
    json.dump(user, f, indent=4)

## Train the model

## A. Decision Tree

In [ ]:
from sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import export_graphviz
import graphviz
import pydotplus
from IPython.display import Image, display
import json
import pickle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

In [ ]:
# Import the user's data
with open('user.json', "r") as f:
        user=json.load(f)

In [ ]:
# Tree creation
dtc = tree.DecisionTreeClassifier()

# Label encoding creation
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()
le5 = LabelEncoder()
le6 = LabelEncoder()
le7 = LabelEncoder()
le8 = LabelEncoder()

### __A.1 Create the model__

In [ ]:
# Dataframe creation
data = user["data"]
result = user["result"]
dataframe = pd.DataFrame(data, columns=["Type", "Attribute", "Card-set", "Caracteristic", "Level", "color", "Rarity"])
resultframe = pd.DataFrame(result, columns=["favorite"])

Type=choice(Type_t) 
Attribute=choice(Attribute_t) 
Rarity=choice(Rarity_t) 
Set=choice(Set_t) 
Caracteristic=choice(Caracteristic_t) 
Level=choice(Level_t)

# Data encoding
dataframe["Type"] = le1.fit_transform(dataframe["Type"])
dataframe["Attribute"] = le2.fit_transform(dataframe["Attribute"])
dataframe["Card-set"] = le3.fit_transform(dataframe["Card-set"])
dataframe["Rarity"] = le4.fit_transform(dataframe["Rarity"])
dataframe["Caracteristic"] = le5.fit_transform(dataframe["Caracteristic"])
dataframe["Level"] = le6.fit_transform(dataframe["Level"])
dataframe["color"] = le7.fit_transform(dataframe["color"])
resultframe["favorite"] = le8.fit_transform(resultframe["favorite"])

# Train the model
dtc.fit(dataframe, resultframe)

### __A.2 Model Vizualisation__

In [ ]:
# Tree visualization
dot_data = tree.export_graphviz(
    dtc,
    out_file=None,
    feature_names=dataframe.columns,
    filled=True,
    rounded=True,
    class_names=le8.inverse_transform(resultframe.favorite.unique()),
    special_characters=True,
)
graph = graphviz.Source(dot_data)

pydot_graph = pydotplus.graph_from_dot_data(dot_data)
img = Image(pydot_graph.create_png())
display(img) 

### __A.3 Evaluate the model__

In [ ]:
# Model accuracy
print("Accuracy:",metrics.accuracy_score(resultframe, dtc.predict(dataframe)))

# F1 score of the model
print("F1 score:",metrics.f1_score(resultframe, dtc.predict(dataframe)))


In [ ]:
# Model prediction
y_pred = dtc.predict(dataframe)

# Creating confusion matrix
cm = confusion_matrix(resultframe, y_pred, normalize='true')
    
# Displaying confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['not favorite', 'favorite'])
disp.plot(cmap='Blues')
plt.show()

### __Save the model__

In [ ]:
pickle.dump(dtc, open("decision_tree.pkl", "wb"))

## B. Random Forest

### __B.1 Create the Model__

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import json
from sklearn.datasets import make_classification
from sklearn.tree import plot_tree
import pickle
from sklearn.metrics import f1_score

In [ ]:
# Import the user's data
with open('user.json', 'r') as f:
    user_data = json.load(f)

# Label encoding creation
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()
le5 = LabelEncoder()
le6 = LabelEncoder()
le7 = LabelEncoder()
le8 = LabelEncoder()

In [ ]:
# Dataframe creation
data = user_data["data"]
result = user_data["result"]
dataframe = pd.DataFrame(data, columns=["Type", "Attribute", "Card-set", "Caracteristic", "Level", "color", "Rarity"])
resultframe = pd.DataFrame(result, columns=["favorite"])

# Data encoding
dataframe["Type"] = le1.fit_transform(dataframe["Type"])
dataframe["Attribute"] = le2.fit_transform(dataframe["Attribute"])
dataframe["Card-set"] = le3.fit_transform(dataframe["Card-set"])
dataframe["Rarity"] = le4.fit_transform(dataframe["Rarity"])
dataframe["Caracteristic"] = le5.fit_transform(dataframe["Caracteristic"])
dataframe["Level"] = le6.fit_transform(dataframe["Level"])
dataframe["color"] = le7.fit_transform(dataframe["color"])
resultframe["favorite"] = le8.fit_transform(resultframe["favorite"])

# Split the data into training and test data
train_data = dataframe.sample(frac=0.8, random_state=0)
test_data = dataframe.drop(train_data.index)

# split the labels into training and test labels
train_labels = resultframe.sample(frac=0.8, random_state=0)
test_labels = resultframe.drop(train_labels.index)

# Define the model
model = RandomForestClassifier(n_estimators=7, random_state=0)

# Train the model
model.fit(train_data, train_labels)

### __B.2 Model Vizualisation__

In [ ]:
plot_tree(model.estimators_[0], filled=True)

### __B.3 Evaluate the model__

In [ ]:
# Model accuracy
y_pred = model.predict(train_data)
print("Accuracy:", accuracy_score(train_labels, y_pred))

# F1 score of the model
print("F1 score:", f1_score(train_labels, y_pred, average='macro'))

In [ ]:
# Displaying confusion matrix
cm = confusion_matrix(train_labels, y_pred, normalize='true')

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['not favorite', 'favorite'])
disp.plot(cmap='Blues')
plt.show()

### __Save the model__

In [ ]:
# Save the model
pickle.dump(model, open("random_forest.pkl", "wb"))

## Recommendation system

In [ ]:
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
import pickle
import json
import random
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def visualisation(tag,value_tag):
    # Load the data from the JSON file
    with open("database.json") as f:
        data = json.load(f)
    # Convert the data to a pandas DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')

    # Unpack the "tags" dictionary into columns
    df = pd.concat([df.drop('tags', axis=1), df['tags'].apply(pd.Series)], axis=1)

    # Group the DataFrame by "Type 1"
    grouped = df.groupby(tag)

    # Print the size of each group
    ax=grouped.size().plot(kind="bar", title=f"Nombre d'images par {tag}")
    # Color the bar(s) corresponding to the value_tag in red
    for i, label in enumerate(ax.get_xticklabels()):
        if label.get_text() == value_tag:
            ax.patches[i].set_color('yellow')

In [ ]:
def recommend_yugioh():
    satisfait = False # Variable de controle de la boucle

    # import data
    with open('database.json', 'r') as f:
        data = json.load(f)

    with open('recommendation.json', 'r') as f:
        rec= json.load(f)

    # import model
    random_forest = pickle.load(open("random_forest.pkl", "rb"))
    decision_tree = pickle.load(open("decision_tree.pkl", "rb"))

    # Label encoding creation
    le1 = LabelEncoder()
    le2 = LabelEncoder()
    le3 = LabelEncoder()
    le4 = LabelEncoder()
    le5 = LabelEncoder()
    le6 = LabelEncoder()
    le7 = LabelEncoder()

    recommandations = []
    

    while not satisfait:

        # User input
        print("Hello, I am a yugioh assistant, I will help you choose your next yu gi oh card")
        type = input("What is the type you like the most: Monster, Trap or Spell ? ")
        attribute = input("What is the attribute you like the most: " + str(rec["Attribute"]) + " ? ")
        card_set = input("What is the card_set you like the most: " + str(rec["Set"]) + " ? ")
        rarity = input("What is the rarity you like the most: " + str(rec["Rarity"]) + " ? ")
        caracteristic = input("What is the caracteristic you like the most: " + str(rec["Caracteristic"]) + " ? ")
        level = input("What is the level you like the most: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 ? ")
        couleur = input("What is the color you like the most: " + str(rec["Color"]) + " ? ")

        # # Random input
        # type = 'Monster'
        # attribute = random.choice(rec["Attribute"])
        # card_set = random.choice(rec["Set"])
        # rarity = random.choice(rec["Rarity"])
        # caracteristic = random.choice(rec["Caracteristic"])
        # level = 4
        # couleur = random.choice(rec["Color"])

        dataframe = pd.DataFrame([[type, attribute, card_set, caracteristic, level, couleur, rarity]], columns=["Type", "Attribute", "Card-set", "Caracteristic", "Level", "color", "Rarity"])

        # Data encoding
        dataframe["Type"] = le1.fit_transform(dataframe["Type"])
        dataframe["Attribute"] = le2.fit_transform(dataframe["Attribute"])
        dataframe["Card-set"] = le3.fit_transform(dataframe["Card-set"])
        dataframe["Rarity"] = le4.fit_transform(dataframe["Rarity"])
        dataframe["Caracteristic"] = le5.fit_transform(dataframe["Caracteristic"])
        dataframe["Level"] = le6.fit_transform(dataframe["Level"])
        dataframe["color"] = le7.fit_transform(dataframe["color"])
            
        # Prediction
        prediction1 = random_forest.predict(dataframe)
        prediction3 = decision_tree.predict(dataframe)

        # If 2 of the 3 models predict that the Yu Gi Oh card will be favorite, then the Yu Gi Oh card will be favorite
        if (prediction1[0] == 0 or prediction3[0] == 0):
            print("The card will be favorite")

            # Add the recommended Yu Gi Oh cards to recommandations
            for yugioh_data in data.values():
                if yugioh_data["tags"] != '':
                    if (yugioh_data["couleur dominante"] == couleur or
                        ((yugioh_data["tags"]["Card type"] == type or
                        yugioh_data["tags"]["Attribute"] == attribute or
                        yugioh_data["tags"]["Card Set"] == card_set or
                        yugioh_data["tags"]["Rarity"] == caracteristic or
                        yugioh_data["tags"]["Caracteristic"] == caracteristic or
                        yugioh_data["tags"]["Level"] == level))) :
                        recommandations.append(yugioh_data)
                    else:
                        if (yugioh_data["couleur dominante"] == couleur and
                            yugioh_data["tags"]["Card Set"] == card_set):
                            recommandations.append(yugioh_data)        
            satisfait = True
        else:
            # Else, restart
            print("The Yu Gi Oh card will not be favorite")
            satisfait = False

    # Display the recommended Yu Gi Oh cards
    card_limit = 3
    counter = 0
    print("The recommended Yu Gi Oh cards are:")
    if recommandations==[]:
        print("No Yu Gi Oh card corresponds to your criteria")
    else:
        for i in range(3):

            yugioh = random.choice(recommandations)
            
            # find images using the name of the card to find the id
            fig, axes = plt.subplots(1, 2, figsize=(10, 5))
            img = Image.open("./Yugi_images/" + str(yugioh['id']) + ".png")
            
            counter += 1
            if yugioh["tags"] != '':
                if yugioh["tags"]["Card type"] == 'Monster':
                    if yugioh["tags"]["Level"] != None:
                        text = "Nom: " + yugioh["tags"]["Card Name"] + "\nType :" + yugioh["tags"]["Attribute"] +  "\nCard-set: " + yugioh["tags"]["Card Set"] + "\nrarity: " + yugioh["tags"]["Rarity"] + "\nCaracteristic: " + yugioh["tags"]["Caracteristic"] + "\nLevel: " + str(yugioh["tags"]["Level"]) + "\nCouleur dominante: " + yugioh["couleur dominante"]
                    else:
                        text = "Nom: " + yugioh["tags"]["Card Name"] + "\nType :" + yugioh["tags"]["Attribute"] +  "\nCard-set: " + yugioh["tags"]["Card Set"] + "\nrarity: " + yugioh["tags"]["Rarity"] + "\nCaracteristic: " + yugioh["tags"]["Caracteristic"] + "\nCouleur dominante: " + yugioh["couleur dominante"]
                else :
                    text = "Nom: " + yugioh["tags"]["Card Name"] + "\nType :" + yugioh["tags"]["Card type"] +  "\nCard-set: " + "\nrarity: " + yugioh["tags"]["Rarity"] + yugioh["tags"]["Card Set"] + "\nCouleur dominante: " + yugioh["couleur dominante"]
            else:
                print
                text = "Nom: " + yugioh["tags"]["Card Name"] + "\nCouleur dominante: " + "\nrarity: " + yugioh["tags"]["Rarity"] + yugioh["couleur dominante"]

            axes[0].imshow(img)
            axes[1].text(0.5, 0.5,text,fontsize=14, ha='center', va='center')
            axes[0].axis('off')
            axes[1].axis('off')
            plt.show()

            if yugioh["tags"] != '':
                visualisation("Card type",yugioh["tags"]["Card type"])
                plt.show()
                if yugioh["tags"]["Card type"] == 'Monster':
                    if yugioh["tags"]["Level"] != None:
                        visualisation("Level",yugioh["tags"]["Level"])
                        plt.show()
                    visualisation("Attribute",yugioh["tags"]["Attribute"])
                    plt.show()
                    visualisation("Caracteristic",yugioh["tags"]["Caracteristic"])
                    plt.show()
                visualisation("Rarity",yugioh["tags"]["Rarity"])
                plt.show()
                visualisation("couleur dominante",yugioh["couleur dominante"])
                plt.show()

            if counter == card_limit:
                break

## Visualisation des données

In [ ]:
recommend_yugioh()

## Tests

In [ ]:
# Test des 2 modèles sur 3 cartes

# {'id': 'JOTL-EN053 R', 'format': 'PNG', 'size': [385, 568], 'orientation': 'portrait', 'creation_date': '16/03/2023', 'tags': {'Card Name': 'Number 102: Star Seraph Sentry', 'Card Set': 'Judgment of the Light', 'Rarity': 'Rare', 'Card type': 'Monster', 'Attribute': 'LIGHT', 'Caracteristic': 'Fairy', 'Level': None}, 'couleur dominante': 'dimgrey'}
# {'id': 'CRMS-EN088 SR', 'format': 'PNG', 'size': [400, 580], 'orientation': 'portrait', 'creation_date': '16/03/2023', 'tags': {'Card Name': 'Code A Ancient Ruins', 'Card Set': 'Crimson Crisis', 'Rarity': 'Super Rare', 'Card type': 'Spell', 'Attribute': None, 'Caracteristic': None, 'Level': None}, 'couleur dominante': 'darkslategrey'}
# {'id': 'LODT-EN001 ScR', 'format': 'PNG', 'size': [400, 580], 'orientation': 'portrait', 'creation_date': '16/03/2023', 'tags': {'Card Name': 'Honest', 'Card Set': 'Light of Destruction', 'Rarity': 'Secret Rare', 'Card type': 'Monster', 'Attribute': 'LIGHT', 'Caracteristic': 'Fairy', 'Level': 4.0}, 'couleur dominante': 'darkolivegreen'}
# {'id': 'MRD-056 R', 'format': 'PNG', 'size': [475, 681], 'orientation': 'portrait', 'creation_date': '16/03/2023', 'tags': {'Card Name': 'Fake Trap', 'Card Set': 'Metal Raiders', 'Rarity': 'Rare', 'Card type': 'Trap', 'Attribute': None, 'Caracteristic': None, 'Level': None}, 'couleur dominante': 'darkslategrey'}
# {'id': 'STON-EN007 C', 'format': 'JPEG', 'size': [400, 580], 'orientation': 'portrait', 'creation_date': '16/03/2023', 'tags': {'Card Name': 'The Six Samurai - Yaichi', 'Card Set': 'Strike of Neos', 'Rarity': 'Common', 'Card type': 'Monster', 'Attribute': 'WATER', 'Caracteristic': 'Warrior', 'Level': 3.0}, 'couleur dominante': 'darkslategrey'}
# {'id': 'PGD-049 C', 'format': 'JPEG', 'size': [400, 580], 'orientation': 'portrait', 'creation_date': '16/03/2023', 'tags': {'Card Name': 'Trap Dustshoot', 'Card Set': 'Pharaonic Guardian', 'Rarity': 'Common', 'Card type': 'Trap', 'Attribute': None, 'Caracteristic': None, 'Level': None}, 'couleur dominante': 'black'}
# {'id': 'EXVC-EN031 C', 'format': 'JPEG', 'size': [400, 580], 'orientation': 'portrait', 'creation_date': '16/03/2023', 'tags': {'Card Name': 'Karakuri Ninja mdl 7749 "Nanashick"', 'Card Set': 'Extreme Victory', 'Rarity': 'Common', 'Card type': 'Monster', 'Attribute': 'EARTH', 'Caracteristic': 'Machine', 'Level': 5.0}, 'couleur dominante': 'darkslategrey'}
# {'id': 'AST-016 R', 'format': 'PNG', 'size': [400, 580], 'orientation': 'portrait', 'creation_date': '16/03/2023', 'tags': {'Card Name': 'Avatar of The Pot', 'Card Set': 'Ancient Sanctuary', 'Rarity': 'Rare', 'Card type': 'Monster', 'Attribute': 'EARTH', 'Caracteristic': 'Rock', 'Level': 3.0}, 'couleur dominante': 'darkolivegreen'}
# {'id': 'CHIM-EN074 R', 'format': 'PNG', 'size': [475, 694], 'orientation': 'portrait', 'creation_date': '16/03/2023', 'tags': {'Card Name': 'Blessed Winds', 'Card Set': 'Chaos Impact', 'Rarity': 'Rare', 'Card type': 'Trap', 'Attribute': None, 'Caracteristic': None, 'Level': None}, 'couleur dominante': 'dimgrey'}


# On choisit 3 cartes au hasard
carte1 = {'Type': 'Monster', 'Attribute': 'LIGHT', 'Card-set': 'Judgment of the Light', 'Caracteristic': 'Fairy', 'Level': None, 'color': 'dimgrey', 'Rarity': 'Rare'}
carte2 = {'Type': 'Spell', 'Attribute': None, 'Card-set': 'Crimson Crisis', 'Caracteristic': None, 'Level': None, 'color': 'darkslategrey', 'Rarity': 'Super Rare'}
carte3 = {'Type': 'Monster', 'Attribute': 'EARTH', 'Card-set': 'Sanctuary', 'Caracteristic': 'Rock', 'Level': 4.0, 'color': 'darkolivegreen', 'Rarity': 'Rare '}

# Créer un dataframe pour le nouveau Pokémon
yugioh = pd.DataFrame([carte1], columns=["Type", "Attribute", "Card-set", "Caracteristic", "Level", "color", "Rarity"])
yugioh2 = pd.DataFrame([carte2], columns=["Type", "Attribute", "Card-set", "Caracteristic", "Level", "color", "Rarity"])
yugioh3 = pd.DataFrame([carte3], columns=["Type", "Attribute", "Card-set", "Caracteristic", "Level", "color", "Rarity"])

# Label encoding creation
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()
le5 = LabelEncoder()
le6 = LabelEncoder()
le7 = LabelEncoder()

# Encoder les données
yugioh["Type"] = le1.fit_transform(yugioh["Type"])
yugioh["Attribute"] = le2.fit_transform(yugioh["Attribute"])
yugioh["Card-set"] = le3.fit_transform(yugioh["Card-set"])
yugioh["Rarity"] = le4.fit_transform(yugioh["Rarity"])
yugioh["Caracteristic"] = le5.fit_transform(yugioh["Caracteristic"])
yugioh["Level"] = le6.fit_transform(yugioh["Level"])
yugioh["color"] = le7.fit_transform(yugioh["color"])

yugioh2["Type"] = le1.fit_transform(yugioh2["Type"])
yugioh2["Attribute"] = le2.fit_transform(yugioh2["Attribute"])
yugioh2["Card-set"] = le3.fit_transform(yugioh2["Card-set"])
yugioh2["Rarity"] = le4.fit_transform(yugioh2["Rarity"])
yugioh2["Caracteristic"] = le5.fit_transform(yugioh2["Caracteristic"])
yugioh2["Level"] = le6.fit_transform(yugioh2["Level"])
yugioh2["color"] = le7.fit_transform(yugioh2["color"])

yugioh3["Type"] = le1.fit_transform(yugioh3["Type"])
yugioh3["Attribute"] = le2.fit_transform(yugioh3["Attribute"])
yugioh3["Card-set"] = le3.fit_transform(yugioh3["Card-set"])
yugioh3["Rarity"] = le4.fit_transform(yugioh3["Rarity"])
yugioh3["Caracteristic"] = le5.fit_transform(yugioh3["Caracteristic"])
yugioh3["Level"] = le6.fit_transform(yugioh3["Level"])
yugioh3["color"] = le7.fit_transform(yugioh3["color"])


# importation des modèles
random_forest = pickle.load(open("random_forest.pkl", "rb"))
decision_tree = pickle.load(open("decision_tree.pkl", "rb"))

# Prédire si le Pokémon sera favori ou non
prediction1 = random_forest.predict(yugioh)
prediction3 = decision_tree.predict(yugioh)

prediction4 = random_forest.predict(yugioh2)
prediction6 = decision_tree.predict(yugioh2)

prediction7 = random_forest.predict(yugioh3)
prediction9 = decision_tree.predict(yugioh3)


# Afficher la prédiction
print("Pour la carte 1 :")
if prediction1[0]==0 :
    print("d'apres le random forest le Pokémon sera favori")
else:
    print("d'apres le random forest le Pokémon ne sera pas favori")

if prediction3[0]==0 :
    print("d'apres le decision tree le Pokémon sera favori")
else:
    print("d'apres le decision tree le Pokémon ne sera pas favori")

print("Pour la carte 2 :")
if prediction4[0]==0 :
    print("d'apres le random forest le Pokémon sera favori")
else:
    print("d'apres le random forest le Pokémon ne sera pas favori")

if prediction6[0]==0 :
    print("d'apres le decision tree le Pokémon sera favori")
else:
    print("d'apres le decision tree le Pokémon ne sera pas favori")

print("Pour la carte 3 :")
if prediction7[0]==0 :
    print("d'apres le random forest le Pokémon sera favori")
else:
    print("d'apres le random forest le Pokémon ne sera pas favori")

if prediction9[0]==0 :
    print("d'apres le decision tree le Pokémon sera favori")
else:
    print("d'apres le decision tree le Pokémon ne sera pas favori")
